In [1]:
import re
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import requests
from bs4 import BeautifulSoup

In [5]:
df = pd.read_csv(r'C:\Users\sahil\Desktop\IPL Project\Data\IPL_SCHEDULE_2008_2020.csv')
df.head()

,id,IPL_year,Match_Info,Match_Date,Match_Team,Match_Result,Match_Cricbuzz_URL,URL,Stadium,Location,Highlights_available
0,5686059.0,2008,1st match,18-04-2008,ROYAL CHALLENGERS BANGALORE vs KOLKATA KNIGHT ...,Kolkata Knight Riders won by 140 runs,https://www.cricbuzz.com/cricket-scores/10557/...,https://www.cricbuzz.com/live-cricket-scorecar...,M.Chinnaswamy Stadium,Bengaluru,False
1,7762380.0,2008,2nd match,19-04-2008,KINGS XI PUNJAB vs CHENNAI SUPER KINGS,Chennai Super Kings won by 33 runs,https://www.cricbuzz.com/cricket-scores/10556/...,https://www.cricbuzz.com/live-cricket-scorecar...,Punjab Cricket Association IS Bindra Stadium,Mohali,False
2,1491263.0,2008,3rd match,19-04-2008,DELHI CAPITALS vs RAJASTHAN ROYALS,Delhi Daredevils won by 9 wickets (with 29 bal...,https://www.cricbuzz.com/cricket-scores/10555/...,https://www.cricbuzz.com/live-cricket-scorecar...,Arun Jaitley Stadium,Delhi,False
3,7662859.0,2008,4th match,20-04-2008,KOLKATA KNIGHT RIDERS vs DECCAN CHARGERS,Kolkata Knight Riders won by 5 wickets (with 6...,https://www.cricbuzz.com/cricket-scores/10554/...,https://www.cricbuzz.com/live-cricket-scorecar...,Eden Gardens,Kolkata,False
4,8361472.0,2008,5th match,20-04-2008,MUMBAI INDIANS vs ROYAL CHALLENGERS BANGALORE,Royal Challengers Bangalore won by 5 wickets (...,https://www.cricbuzz.com/cricket-scores/10553/...,https://www.cricbuzz.com/live-cricket-scorecar...,Wankhede Stadium,Mumbai,False


In [8]:
failed_urls = []

# Assuming your dataframe is already loaded into 'df' and it contains a column 'URL' with match URLs.
# file_name = 'cricket_scorecards.xlsx'

# Loop through each URL in the 'df' DataFrame
for url in df['URL']:
    # Fetch the data from the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract scorecard data
    scorecard = soup.find_all('div', class_='cb-col cb-col-100 cb-ltst-wgt-hdr')
    temp = ''
    for section in scorecard:
        temp += section.get_text(separator='\n')
    temp = temp.replace("\n", "")

    # Extract match_id and year from the URL
    match_id = int(re.search(r'/(\d+)/', url).group(1))
    year = int(re.search(r'(\d+)$', url).group(1))

    # Extract team names
    teams = re.findall(r'([A-Za-z\s]+) Innings', temp)
    teams = [team.strip() for team in teams]

    # Check if we extracted exactly two teams
    if len(teams) != 2:
        print(f"Could not extract team names for match {match_id}. URL: {url}")
        failed_urls.append(url)
        continue  # Skip to the next URL

    # Split the scorecard into innings
    innings = temp.split(f"{teams[1]} Innings")
    innings_data = [innings[0], innings[1]]

    # Create empty lists for storing data
    batters_data = []
    bowlers_data = []
    extras_data = []

    # Regex patterns
    batters_regex = r'([A-Za-z\s]+)\s+(not out|run out|c [A-Za-z\s]+ b [A-Za-z\s]+|st [A-Za-z\s]+ b [A-Za-z\s]+|b [A-Za-z\s]+|lbw b [A-Za-z\s]+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+([\d.]+)'
    bowlers_regex = r'([A-Za-z\s]+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+([\d.]+)'
    extras_pattern = r'Extras\s+(\d+)\s+\(b (\d+), lb (\d+), w (\d+), nb (\d+), p (\d+)\)'
    total_pattern = r'Total\s+(\d+)\s+\((\d+)\s+wkts?,\s+(\d+(\.\d+)?)\s+Ov\)'

    # Loop through the innings and extract relevant data
    for i, inning in enumerate(innings_data):
        try:
            inning = re.sub(r'^.*?R B 4s 6s SR\s*', '', inning)
            batting_team = teams[i]
            bowling_team = teams[1 - i]

            # Extract batters data
            batters = re.findall(batters_regex, inning)
            for batter in batters:
                batter_name_with_dismissal = f"{batter[0].strip()} {batter[1].strip()}"
                batters_data.append({
                    'Match ID': match_id,
                    'Year': year,
                    'Team': batting_team,
                    'Batter': batter_name_with_dismissal,
                    'R': int(batter[2]),
                    'B': int(batter[3]),
                    '4s': int(batter[4]),
                    '6s': int(batter[5]),
                    'SR': float(batter[6]),
                })

            # Extract extras data
            extras_match = re.search(extras_pattern, inning)
            if extras_match:
                extras_data.append({
                    'Match ID': match_id,
                    'Year': year,
                    'Team': bowling_team,
                    'Total_extras': int(extras_match.group(1)),
                    'Byes': int(extras_match.group(2)),
                    'Legbyes': int(extras_match.group(3)),
                    'Wides': int(extras_match.group(4)),
                    'Noballs': int(extras_match.group(5)),
                    'Penalty_runs': int(extras_match.group(6)),
                })
            else:
                print(f"Extras data not found for match {match_id}.")

            # Extract total data
            total_match = re.search(total_pattern, inning)
            if total_match:
                extras_data[i]['Total_Score'] = int(total_match.group(1))
                extras_data[i]['Total_wkts'] = int(total_match.group(2))
                extras_data[i]['Total_Overs'] = float(total_match.group(3))
            else:
                print(f"Total data not found for match {match_id}.")

            # Extract bowlers data
            inning = re.sub(r'^.*?Bowler O M R W NB WD ECO\s*', '', inning)
            bowlers = re.findall(bowlers_regex, inning)
            for bowler in bowlers:
                bowlers_data.append({
                    'Match ID': match_id,
                    'Year': year,
                    'Team': bowling_team,
                    'Bowler': bowler[0].strip(),
                    'O': int(bowler[1]),
                    'M': int(bowler[2]),
                    'R': int(bowler[3]),
                    'W': int(bowler[4]),
                    'NB': int(bowler[5]),
                    'WD': int(bowler[6]),
                    'ECO': float(bowler[7]),
                })

        except IndexError as e:
            print(f"Index error occurred for match ID {match_id} while processing innings {i}: {e}")
            failed_urls.append(url)
            continue
        except Exception as e:
            print(f"An error occurred for match ID {match_id}: {e}")
            failed_urls.append(url)
            continue

    # Swap the Total_Score and Total_Overs between teams if necessary
    if len(extras_data) == 2:
        extras_data[1]['Total_Score'], extras_data[0]['Total_Score'] = extras_data[0]['Total_Score'], extras_data[1]['Total_Score']
        extras_data[1]['Total_Overs'], extras_data[0]['Total_Overs'] = extras_data[0]['Total_Overs'], extras_data[1]['Total_Overs']

    # Convert lists to DataFrames
    batters_df = pd.DataFrame(batters_data)
    bowlers_df = pd.DataFrame(bowlers_data)
    extras_df = pd.DataFrame(extras_data)

    # Append data to the Excel file
    with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        start_row = writer.sheets['Batters'].max_row  # Get the last row of the sheet
        batters_df.to_excel(writer, sheet_name='Batters', startrow=start_row, header=False, index=False)
        start_row = writer.sheets['Bowlers'].max_row
        bowlers_df.to_excel(writer, sheet_name='Bowlers', startrow=start_row, header=False, index=False)
        start_row = writer.sheets['Extras'].max_row
        extras_df.to_excel(writer, sheet_name='Extras', startrow=start_row, header=False, index=False)

    print(f"Data for match ID {match_id} from {url} appended to '{file_name}' successfully!")


Data for match ID 10557 from https://www.cricbuzz.com/live-cricket-scorecard/10557/rcb-vs-kkr-1st-match-indian-premier-league-2008 appended to 'cricket_scorecards.xlsx' successfully!
Data for match ID 10556 from https://www.cricbuzz.com/live-cricket-scorecard/10556/kxip-vs-csk-2nd-match-indian-premier-league-2008 appended to 'cricket_scorecards.xlsx' successfully!
Data for match ID 10555 from https://www.cricbuzz.com/live-cricket-scorecard/10555/dc-vs-rr-3rd-match-indian-premier-league-2008 appended to 'cricket_scorecards.xlsx' successfully!
Data for match ID 10554 from https://www.cricbuzz.com/live-cricket-scorecard/10554/kkr-vs-dcg-4th-match-indian-premier-league-2008 appended to 'cricket_scorecards.xlsx' successfully!
Data for match ID 10553 from https://www.cricbuzz.com/live-cricket-scorecard/10553/mi-vs-rcb-5th-match-indian-premier-league-2008 appended to 'cricket_scorecards.xlsx' successfully!
Data for match ID 10552 from https://www.cricbuzz.com/live-cricket-scorecard/10552/rr-v

In [9]:
failed_urls

['https://www.cricbuzz.com/live-cricket-scorecard/10511/dc-vs-kkr-47th-match-indian-premier-league-2008',
 'https://www.cricbuzz.com/live-cricket-scorecard/10599/mi-vs-rr-7th-match-indian-premier-league-2009',
 'https://www.cricbuzz.com/live-cricket-scorecard/10593/csk-vs-kkr-13th-match-indian-premier-league-2009',
 'https://www.cricbuzz.com/live-cricket-scorecard/9917/rcb-vs-rr-20th-match-indian-premier-league-2011',
 'https://www.cricbuzz.com/live-cricket-scorecard/9966/dc-vs-pwi-68th-match-indian-premier-league-2011',
 'https://www.cricbuzz.com/live-cricket-scorecard/11194/kxip-vs-pwi-14th-match-indian-premier-league-2012',
 'https://www.cricbuzz.com/live-cricket-scorecard/11198/kkr-vs-kxip-17th-match-indian-premier-league-2012',
 'https://www.cricbuzz.com/live-cricket-scorecard/11206/kxip-vs-rcb-25th-match-indian-premier-league-2012',
 'https://www.cricbuzz.com/live-cricket-scorecard/11213/kkr-vs-dcg-32nd-match-indian-premier-league-2012',
 'https://www.cricbuzz.com/live-cricket-sc

In [10]:
len(failed_urls)

32